<a href="https://colab.research.google.com/github/pabloudea/AstronomiaPractica1/blob/master/KeplerEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
%pylab inline
import matplotlib.pyplot as plt

#Using NumPy's libraries.
pi, sin, cos, tan, sqrt, arctan = np.pi, np.sin, np.cos, np.tan, np.sqrt, np.arctan

Populating the interactive namespace from numpy and matplotlib


# Ecuación de Kepler
## Astronomía Práctica I - UdeA

Esta página contiene el desarrollo realizado por Pablo Restrepo Valencia de la [práctica](https://github.com/pabloudea/AstronomiaPractica1/blob/master/II-211%20ECUACION%20DE%20KEPLER%20200510.pdf) sobre la solución a la [ecuación de Kepler](https://en.wikipedia.org/wiki/Kepler%27s_equation), construida por el profesor Ignacio Ferrín para el curso de astronomía práctica 1. 

Se desarrolla el algoritmo del método iterativo, descrito en la sección 3.c de la guía, para hallar la anomalía excentrica $E$ a partir de la [excentricidad]() $e$ y la [anomalía media]() $M$.




Primero, definimos la función llamada $kepler(M,e,E) = M - E + e*sinE$, sobre la cual aplicaremos el método iterativo.

Adicionalmente, se definen las funciones $anomaly$ y $radio$ para hallar la anomalía verdadera $\theta$ y la distancia radial del foco al objeto $r$, respectivamente. &emsp;(Ver Ecuaciones $16-17$ de la guía.)




In [0]:
#Definition of Kepler Equation
def kepler(M,e,E):
    func = M-E+e*sin(E)
    return func

#Formula to calculate the true anomaly from the eccentric anomaly E and the eccentricity e.
def anomaly(e,E):
    anom = 2*arctan(sqrt((1+e)/(1-e))*tan(E/2))
    return anom

#Formula to calculate the radius of the body. It's necessary know the semimajor axis a.
radio = lambda a,e,E: a*(1 - e*cos(E))

Una vez definidas dichas fórmulas, pasamos a desarrollar propiamente el código para el método iterativo. Este tiene como parámetros de entrada $M$, $e$, $\Delta E$ y la precisión deseada $C$. Devuelve un arreglo con los valores finales para $E$, $\theta$ y $r$, expresados en radianes $[rad]$ y unidades astronómicas $AU$. 


Además, para poder hallar la distancia radial $r$, se ingresa en el método un valor fijo para el semieje mayor de la órbita $a = 1.0$ (el cual puede ser modificado a la hora de llamar la función). También se incluye la opción de imprimir los resultados de cada iteración, como se muestra en las tablas $1-3$ de la sección 3.c $(printer)$ y la tabla con los resultados como se piden en la tarea 2, que a su vez, incluye los datos solicitados en la tarea 1 de la guía $(tarea)$. Estas opciones se omiten por defecto. $(printer = tarea = False)$.

Como se puede ver en el diagrama de flujo de la figura 3 de la guía, el método consiste de dos ciclos, uno incrustado dentro del otro. El ciclo interno se encarga de encontrar cuando la función $kepler$ cambia de signo, mientras el externo genera la precisión definida por $C$.






In [0]:
#Interactive Method defined in the practice by Ignacio Ferrín.
#--------------------------------------------------------------
#
def Interactive(M,e,deltaE,C,a=1.0,printer=False, tarea=False):
    """Find the eccentric anomaly from Kepler's equation.
    M: Mean anomaly, e: eccentricity, deltaE: Initial change, 
    a: Semi-major axis (1.0 AU as default), printer:Print the step"""
    
    E = np.array([])
    r = np.array([])
    theta = np.array([])

    #Choose the initial points
    E0 = 0
    E1 = E0+deltaE
    E = np.append(E,[E0,E1])

    #Convert radians to degrees
    deg = 180./pi
    i=1
    
    #Obtain the desired precision
    while not round(E[-1]-E[-2],7) <= round(C,7):
        if printer or tarea: print("\n Interacción %1d:\n"%(i))
        
        #Find a sign change
        while round(kepler(M,e,E[-1]),7) > 0:
            E0 = E[-1]
            E1 += deltaE
            E = np.append(E,[E1])
            if printer:
                print("E[rad] = %.6f, E[°] = %.6f, sen(E) = %.6f, e*sen(E) = %.6f, M-E+e*sen(E) = %.6f"%(E1,E1*deg,sin(E1),e*sin(E1),kepler(M,e,E1)))
        
        true = round(anomaly(e,E[-1]),6)
        radi = round(radio(a,e,E[-1]),6)
        if tarea: print("\nE[°]= %.6f \nC= E(i)-E(i-1): %.6f \ntheta [°] = %.6f \nr [AU] = %.6f"%(E[-2]*deg, E[-1]-E[-2], true*deg, radi))
        deltaE = deltaE/10
        E1 = E0+deltaE
        i += 1
        E = np.delete(E,-1)
        
    theta = np.append(theta,[true])
    r = np.append( r, [radi] )
    return round(E[-1],6), round(theta[-1],6), round(r[-1],6)


A continuación presentamos los resultados de las tareas, para una órbita con semieje mayor $a = 1.0 AU$, $M = 1.11$ y $e = 0.9$. La tolerancia elegida es de $C = 1.0*10^{-6}$ y el paso en la iteración $\Delta E = 0.5$

Nótese como, a pesar de lo excéntrica de la órbita, el proceso requiere de pocas iteraciones para devolver el resultado.

In [0]:
M = 1.11
e = 0.9
deltaE = 0.5
C = 1E-6

Interactive(M,e,deltaE,C,tarea=True)


 Interacción 1:


E[°]= 85.943669 
C= E(i)-E(i-1): 0.500000 
theta [°] = 163.240489 
r [AU] = 1.374532

 Interacción 2:


E[°]= 108.861981 
C= E(i)-E(i-1): 0.050000 
theta [°] = 162.318084 
r [AU] = 1.333163

 Interacción 3:


E[°]= 111.440291 
C= E(i)-E(i-1): 0.005000 
theta [°] = 162.318084 
r [AU] = 1.333163

 Interacción 4:


E[°]= 111.554883 
C= E(i)-E(i-1): 0.000500 
theta [°] = 162.271216 
r [AU] = 1.331072

 Interacción 5:


E[°]= 111.554883 
C= E(i)-E(i-1): 0.000100 
theta [°] = 162.263710 
r [AU] = 1.330737

 Interacción 6:


E[°]= 111.557175 
C= E(i)-E(i-1): 0.000005 
theta [°] = 162.262679 
r [AU] = 1.330691

 Interacción 7:


E[°]= 111.557432 
C= E(i)-E(i-1): 0.000001 
theta [°] = 162.262679 
r [AU] = 1.330691


(1.947044, 2.832018, 1.330691)